# Canada Immigration 2013
### *Normalized to world population*
#### James Cage April 11, 2019

As a class exercise for Coursera's *Data Visualization with Python*, I worked through an example of creating a choropleth map showing immigration to Canada in 2013. Each country was shaded by total immigration from that country. This is a great map if you are interested in the "flavor" of immigration coming into Candada. That is, immigration from the Canadian perspective. But this map may just be world population map. China and India have the largest populations on earth - is it any surprise that a lot of immigrants come from there? 

What if you want to know more about the attractiveness of Canada from a world perspective? In which countries are people most likely to emmigrate to The Great White North?

To answer this, I obtain world population in 2013, and use it to add the following columns to the Canadian immigration dataset:

* Population: From above
* Likelihood: Equal to (immigration from that country)/(population of that country)

I then create the following visualization

* TBD

Finally, I answer the initial question by creating a choropleth map of Likelihood, along with supporting tables and other visualizations.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

1. [Zero](#0)<br>
2. [Two](#2)<br>
3. [Four](#4) <br>
4. [Six](#6) <br>
</div>
<hr>

# Setup

Material in this section adapted from [a notebook](https://labs.cognitiveclass.ai/tools/jupyterlab/lab/tree/labs/DV0101EN/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb) created by [Alex Aklson](https://www.linkedin.com/in/aklson/) for a course on **Coursera** called *Data Visualization with Python*.

#### Import Libraries

In [1]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

#### Install Folium & Import Library

Note: If Folium is **not installed**, prepend this line to the following cell:

!conda install -c conda-forge folium=0.5.0 --yes

In [2]:
# Un-comment the folloowing line if Folium is not installed
#!conda install -c conda-forge folium=0.5.0 --yes

import folium
print('Folium installed and imported!')

Folium installed and imported!


#### Obtain & Clean Canadian Immigration Information

**NOTE**: The data provided for the class included information for 49 regions that were not in the geojson file (see below). This was due to spelling differences for the country names between the files, as well as outright omissions.  Most of these had very little immigration to Canada in 2013, but there were several major ones (including the UK, Russia, Iran, and Venezuela). I corrected the names for the 7 largest missing countries and saved the file as 'Canada2.xlsx', which I use below.

Now download and import the Canadian immigration dataset using *pandas* `read_excel()` method. **xlrd** is a module which *pandas* requires to read in excel files. Use it to download the dataset and read it into a pandas dataframe:

In [3]:
# If xlrd is not installed, uncomment the following line
#!conda install -c anaconda xlrd --yes

# Source of "raw" Excel file: 'https://ibm.box.com/shared/static/lw190pt9zpy5bd1ptyg2aw15awomz9pu.xlsx'

df_can = pd.read_excel('Canada2.xlsx',
                     sheet_name='Canada by Citizenship',
                     skiprows=range(20),
                     skipfooter=2)

print('Data downloaded and read into a dataframe!')

Data downloaded and read into a dataframe!


If desired, use some or all of the following to take a look at the dataset.

In [4]:
df_can.head()

,Type,Coverage,OdName,AREA,AreaName,REG,RegName,DEV,DevName,1980,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Immigrants,Foreigners,Afghanistan,935,Asia,5501,Southern Asia,902,Developing regions,16,...,2978,3436,3009,2652,2111,1746,1758,2203,2635,2004
1,Immigrants,Foreigners,Albania,908,Europe,925,Southern Europe,901,Developed regions,1,...,1450,1223,856,702,560,716,561,539,620,603
2,Immigrants,Foreigners,Algeria,903,Africa,912,Northern Africa,902,Developing regions,80,...,3616,3626,4807,3623,4005,5393,4752,4325,3774,4331
3,Immigrants,Foreigners,American Samoa,909,Oceania,957,Polynesia,902,Developing regions,0,...,0,0,1,0,0,0,0,0,0,0
4,Immigrants,Foreigners,Andorra,908,Europe,925,Southern Europe,901,Developed regions,0,...,0,0,1,1,0,0,0,0,1,1


In [5]:
print(df_can.shape)

(195, 43)


Clean up data. We will make some modifications to the original dataset to make it easier to create our visualizations. Refer to *Introduction to Matplotlib and Line Plots* and *Area Plots, Histograms, and Bar Plots* notebooks for a detailed description of this preprocessing.

In [6]:
# clean up the dataset to remove unnecessary columns (eg. REG) 
df_can.drop(['AREA','REG','DEV','Type','Coverage'], axis=1, inplace=True)

# let's rename the columns so that they make sense
df_can.rename(columns={'OdName':'Country', 'AreaName':'Continent','RegName':'Region'}, inplace=True)

# for sake of consistency, let's also make all column labels of type string
df_can.columns = list(map(str, df_can.columns))

# add total column
df_can['Total'] = df_can.sum(axis=1)

# years that we will be using in this lesson - useful for plotting later on
years = list(map(str, range(1980, 2014)))
print ('data dimensions:', df_can.shape)

data dimensions: (195, 39)


Take a look at the cleaned database

In [7]:
df_can.head()

,Country,Continent,Region,DevName,1980,1981,1982,1983,1984,1985,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,Total
0,Afghanistan,Asia,Southern Asia,Developing regions,16,39,39,47,71,340,...,3436,3009,2652,2111,1746,1758,2203,2635,2004,58639
1,Albania,Europe,Southern Europe,Developed regions,1,0,0,0,0,0,...,1223,856,702,560,716,561,539,620,603,15699
2,Algeria,Africa,Northern Africa,Developing regions,80,67,71,69,63,44,...,3626,4807,3623,4005,5393,4752,4325,3774,4331,69439
3,American Samoa,Oceania,Polynesia,Developing regions,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,6
4,Andorra,Europe,Southern Europe,Developed regions,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,1,15


In order to create a `Choropleth` map, we need a GeoJSON file that defines the areas/boundaries of the state, county, or country that we are interested in. In our case, a GeoJSON that defines the boundaries of all world countries. We will use the map from the Coursera class and name it **world_countries.json**.

In [8]:
# download countries geojson file
!wget --quiet https://ibm.box.com/shared/static/cto2qv7nx6yq19logfcissyy4euo8lho.json -O world_countries.json
    
print('GeoJSON file downloaded!')

GeoJSON file downloaded!


# Get Population Data, Clean, and Merge

In this step, I obtain the world population data from the [Worldbank Open Data website](https://data.worldbank.org/indicator/sp.pop.totl?end=2013&start=1980). My immediate goal is 2013 data, but I'll go ahead and obtain data for the entire range of the existing data frame (1980-2013).

In [9]:
# I picked the Excel version of the data, because the CSV is zipped. 

df_world_pop = pd.read_excel('http://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=excel',
                     sheet_name='Data',
                     skiprows=range(3))

print('Data downloaded and read into a dataframe!')


Data downloaded and read into a dataframe!


In [11]:
df_world_pop.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,105264.0,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,...,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,35530081.0,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,...,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0,29784193.0,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0,76965.0,NaN


In [12]:
df_world_pop.shape

(264, 63)

Let's get rid of columns we don't need & rename the first column. (The data values are floats, which I don't like, but will address later if it causes problems.)

In [11]:
# As a learning exercise, I use data slicing instead of Python list comphrehensions.

# Generate a list of columns we don't want, and then drop them.

bad_columns = list(df_world_pop.loc[:, 'Country Code':'1979'].columns) + \
              list(df_world_pop.loc[:, '2014':'2018'].columns)

df_world_pop.drop(bad_columns, axis=1, inplace=True)

# Rename the first column to match the column in df_can

df_world_pop.rename(columns={'Country Name':'Country'}, inplace=True)

df_world_pop.head()

,Country,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Aruba,60096.0,60567.0,61345.0,62201.0,62836.0,63026.0,62644.0,61833.0,61079.0,...,98737.0,100031.0,100832.0,101220.0,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0
1,Afghanistan,13248370.0,13053954.0,12749645.0,12389269.0,12047115.0,11783050.0,11601041.0,11502761.0,11540888.0,...,24118979.0,25070798.0,25893450.0,26616792.0,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0
2,Angola,8929900.0,9244507.0,9582156.0,9931562.0,10277321.0,10609042.0,10921037.0,11218268.0,11513968.0,...,18865716.0,19552542.0,20262399.0,20997687.0,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0
3,Albania,2671997.0,2726056.0,2784278.0,2843960.0,2904429.0,2964762.0,3022635.0,3083605.0,3142336.0,...,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0
4,Andorra,36067.0,37500.0,39114.0,40867.0,42706.0,44600.0,46517.0,48455.0,50434.0,...,76244.0,78867.0,80991.0,82683.0,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0


In [27]:
df_can.head(2)

,Continent,Region,DevName,1980,1981,1982,1983,1984,1985,1986,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,Total
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,Asia,Southern Asia,Developing regions,16,39,39,47,71,340,496,...,3436,3009,2652,2111,1746,1758,2203,2635,2004,58639
Albania,Europe,Southern Europe,Developed regions,1,0,0,0,0,0,1,...,1223,856,702,560,716,561,539,620,603,15699


In [20]:
# Set the index for each data frame
df_can.set_index('Country', inplace=True)
df_world_pop.set_index('Country', inplace=True)


KeyError: 'Country'

Now I want to identify any countries in df_can that AREN'T represented in df_world_pop. I get to use Python sets!

First, ARE there any countries in the Canada dataset not found in the World Population Dataset?

In [24]:
can_countries = set(df_can.index)
world_countries = set(df_world_pop.Country)


print("Number of countries in Canada Immigration Dataset: " + str(len(can_countries)))
print("Number of matching countries: " + str(len(can_countries & world_countries)))

Number of countries in Canada Immigration Dataset: 195
Number of matching countries: 166


So yes, 29 countries are "missing", or at least have spelling differences in their names. What are those countries? Can I drop any of these rows?

In [25]:
unmatched = list(can_countries - world_countries)
df_can[df_can.index.isin(unmatched)]





# df[df['A'].isin([3, 6])]
# unmatched
# df_can.iloc['China':'China', :]
# df_can['China']

,Continent,Region,DevName,1980,1981,1982,1983,1984,1985,1986,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,Total
Country,,,,,,,,,,,,,,,,,,,,,
Bahamas,Latin America and the Caribbean,Caribbean,Developing regions,26,23,38,12,21,28,23,...,14,18,23,30,27,25,40,60,55,808
Bolivia (Plurinational State of),Latin America and the Caribbean,South America,Developing regions,44,52,42,49,38,44,79,...,134,139,107,143,214,180,86,83,107,3205
"China, Hong Kong Special Administrative Region",Asia,Eastern Asia,Developing regions,0,0,0,0,0,0,0,...,729,712,674,897,657,623,591,728,774,9327
"China, Macao Special Administrative Region",Asia,Eastern Asia,Developing regions,0,0,0,0,0,0,0,...,21,32,16,12,21,21,13,33,29,284
Congo,Africa,Middle Africa,Developing regions,2,0,1,0,1,2,4,...,136,143,87,72,70,75,81,91,114,1686
Ivory Coast,Africa,Western Africa,Developing regions,14,8,10,6,21,23,23,...,294,431,619,646,668,1066,636,1024,1170,8291
Democratic People's Republic of Korea,Asia,Eastern Asia,Developing regions,1,1,3,1,4,3,0,...,14,10,7,19,11,45,97,66,17,388
Democratic Republic of the Congo,Africa,Middle Africa,Developing regions,15,16,17,17,28,11,38,...,1521,1623,1466,1397,1581,1239,1224,1714,2050,25582
Egypt,Africa,Northern Africa,Developing regions,612,660,755,455,447,348,514,...,2496,2190,2356,3347,3496,5982,4663,5555,4165,72745


In [64]:
df_can.index

Index(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       ...
       'United States of America', 'Uruguay', 'Uzbekistan', 'Vanuatu',
       'Venezuela (Bolivarian Republic of)', 'Viet Nam', 'Western Sahara',
       'Yemen', 'Zambia', 'Zimbabwe'],
      dtype='object', name='Country', length=195)

In [31]:
df_world_pop.head()

,Country,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Aruba,60096.0,60567.0,61345.0,62201.0,62836.0,63026.0,62644.0,61833.0,61079.0,...,98737.0,100031.0,100832.0,101220.0,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0
1,Afghanistan,13248370.0,13053954.0,12749645.0,12389269.0,12047115.0,11783050.0,11601041.0,11502761.0,11540888.0,...,24118979.0,25070798.0,25893450.0,26616792.0,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0
2,Angola,8929900.0,9244507.0,9582156.0,9931562.0,10277321.0,10609042.0,10921037.0,11218268.0,11513968.0,...,18865716.0,19552542.0,20262399.0,20997687.0,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0
3,Albania,2671997.0,2726056.0,2784278.0,2843960.0,2904429.0,2964762.0,3022635.0,3083605.0,3142336.0,...,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0
4,Andorra,36067.0,37500.0,39114.0,40867.0,42706.0,44600.0,46517.0,48455.0,50434.0,...,76244.0,78867.0,80991.0,82683.0,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0


Now I would like to clean the dataframe and make it more compatible with the existing df_can dataframe.

In [60]:
df_world_pop.index

Index(['Aruba', 'Afghanistan', 'Angola', 'Albania', 'Andorra', 'Arab World',
       'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa',
       ...
       'Virgin Islands (U.S.)', 'Vietnam', 'Vanuatu', 'World', 'Samoa',
       'Kosovo', 'Yemen, Rep.', 'South Africa', 'Zambia', 'Zimbabwe'],
      dtype='object', name='Country', length=264)

In [28]:
df_can.head()

,Continent,Region,DevName,1980,1981,1982,1983,1984,1985,1986,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,Total
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,Asia,Southern Asia,Developing regions,16,39,39,47,71,340,496,...,3436,3009,2652,2111,1746,1758,2203,2635,2004,58639
Albania,Europe,Southern Europe,Developed regions,1,0,0,0,0,0,1,...,1223,856,702,560,716,561,539,620,603,15699
Algeria,Africa,Northern Africa,Developing regions,80,67,71,69,63,44,69,...,3626,4807,3623,4005,5393,4752,4325,3774,4331,69439
American Samoa,Oceania,Polynesia,Developing regions,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,6
Andorra,Europe,Southern Europe,Developed regions,0,0,0,0,0,0,2,...,0,1,1,0,0,0,0,1,1,15


# Reference - delete after completion

In [30]:
df_can.tail(2)

,Country,Continent,Region,DevName,1980,1981,1982,1983,1984,1985,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,Total
193,Zambia,Africa,Eastern Africa,Developing regions,11,17,11,7,16,9,...,91,77,71,64,60,102,69,46,59,1677
194,Zimbabwe,Africa,Eastern Africa,Developing regions,72,114,102,44,32,29,...,615,454,663,611,508,494,434,437,407,8598


In [66]:
df_world_pop

,Country,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Aruba,6.009600e+04,6.056700e+04,6.134500e+04,6.220100e+04,6.283600e+04,6.302600e+04,6.264400e+04,6.183300e+04,6.107900e+04,...,9.873700e+04,1.000310e+05,1.008320e+05,1.012200e+05,1.013530e+05,1.014530e+05,1.016690e+05,1.020530e+05,1.025770e+05,1.031870e+05
1,Afghanistan,1.324837e+07,1.305395e+07,1.274964e+07,1.238927e+07,1.204712e+07,1.178305e+07,1.160104e+07,1.150276e+07,1.154089e+07,...,2.411898e+07,2.507080e+07,2.589345e+07,2.661679e+07,2.729403e+07,2.800433e+07,2.880317e+07,2.970860e+07,3.069696e+07,3.173169e+07
2,Angola,8.929900e+06,9.244507e+06,9.582156e+06,9.931562e+06,1.027732e+07,1.060904e+07,1.092104e+07,1.121827e+07,1.151397e+07,...,1.886572e+07,1.955254e+07,2.026240e+07,2.099769e+07,2.175942e+07,2.254955e+07,2.336913e+07,2.421856e+07,2.509615e+07,2.599834e+07
3,Albania,2.671997e+06,2.726056e+06,2.784278e+06,2.843960e+06,2.904429e+06,2.964762e+06,3.022635e+06,3.083605e+06,3.142336e+06,...,3.026939e+06,3.011487e+06,2.992547e+06,2.970017e+06,2.947314e+06,2.927519e+06,2.913021e+06,2.905195e+06,2.900401e+06,2.895092e+06
4,Andorra,3.606700e+04,3.750000e+04,3.911400e+04,4.086700e+04,4.270600e+04,4.460000e+04,4.651700e+04,4.845500e+04,5.043400e+04,...,7.624400e+04,7.886700e+04,8.099100e+04,8.268300e+04,8.386100e+04,8.446200e+04,8.444900e+04,8.375100e+04,8.243100e+04,8.078800e+04
5,Arab World,1.656895e+08,1.710520e+08,1.764901e+08,1.820058e+08,1.876108e+08,1.933103e+08,1.990938e+08,2.049425e+08,2.108448e+08,...,3.091620e+08,3.162647e+08,3.237733e+08,3.316538e+08,3.398255e+08,3.481451e+08,3.565089e+08,3.648959e+08,3.733070e+08,3.817021e+08
6,United Arab Emirates,1.042384e+06,1.120900e+06,1.189545e+06,1.253060e+06,1.318478e+06,1.391052e+06,1.472218e+06,1.560718e+06,1.655849e+06,...,4.087931e+06,4.579562e+06,5.242032e+06,6.044067e+06,6.894278e+06,7.666393e+06,8.270684e+06,8.672475e+06,8.900453e+06,9.006263e+06
7,Argentina,2.810589e+07,2.854336e+07,2.899399e+07,2.945474e+07,2.992090e+07,3.038878e+07,3.085724e+07,3.132647e+07,3.179552e+07,...,3.872870e+07,3.914549e+07,3.955889e+07,3.997022e+07,4.038239e+07,4.079941e+07,4.122389e+07,4.165688e+07,4.209674e+07,4.253992e+07
8,Armenia,3.099751e+06,3.148092e+06,3.193686e+06,3.238594e+06,3.285595e+06,3.335935e+06,3.392256e+06,3.451942e+06,3.504651e+06,...,3.000612e+06,2.981259e+06,2.958500e+06,2.933056e+06,2.908220e+06,2.888584e+06,2.877311e+06,2.875581e+06,2.881922e+06,2.893509e+06
9,American Samoa,3.245700e+04,3.349300e+04,3.473800e+04,3.616000e+04,3.768800e+04,3.924100e+04,4.083700e+04,4.245000e+04,4.404700e+04,...,5.926400e+04,5.911800e+04,5.865000e+04,5.790300e+04,5.703000e+04,5.622700e+04,5.563700e+04,5.532000e+04,5.523000e+04,5.530700e+04
